<a href="https://colab.research.google.com/github/Suhejl/MLDM_Labs/blob/main/week11/L11_Clustering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from matplotlib import pyplot as plt

In [ ]:
RANDOM_SEED = 0x0

# TASK 1 (4 Points): K-Means using scikit-learn

First we will generate some data. The variable `X` will contain the data used in this section.

In [ ]:
from sklearn.datasets import make_blobs

In [ ]:
X, _ = make_blobs(
    n_samples=300,
    centers=4,
    cluster_std=.6,
    random_state=RANDOM_SEED,
)

plt.scatter(X[:, 0], X[:, 1], s=50)

### Task 1a

Cluster the above data using the K-Means implementation provided by scikit-learn.
Refer to the official documentation: <https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html>

* create an instance of the class `sklearn.cluster.KMeans`
* choose a suitable value for the `n_clusters` parameter
* call the `.fit` method to compute the clustering
* call the `.predict` method to get cluster labels for each data point

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
kmeans = KMeans(n_clusters=4, random_state=RANDOM_SEED)
kmeans.fit(X)
labels = kmeans.predict(X)

### Task 1b

Try different numbers of clusters, compute the average silhouette scores using
`sklearn.metrics.silhouette_score` and plot the silhouette plot for the different values of K. Refer to the documentation: <https://scikit-learn.org/stable/modules/generated/sklearn.metrics.silhouette_score.html#sklearn.metrics.silhouette_score>



In [ ]:
from sklearn.metrics import silhouette_score

In [ ]:
Ks = range(2, 11)  # silhouette needs at least 2 clusters
scores = []

for k in Ks:
    km = KMeans(n_clusters=k, random_state=RANDOM_SEED, n_init="auto")
    labels = km.fit_predict(X)              # fit + predict in one call
    scores.append(silhouette_score(X, labels))

plt.figure(figsize=(6, 4))
plt.plot(list(Ks), scores, marker="o")
plt.xticks(list(Ks))
plt.xlabel("Number of clusters K")
plt.ylabel("Average silhouette score")
plt.title("Silhouette score vs. K")
plt.grid(True, alpha=0.3)
plt.show()

### Task 1c

Select the number of clusters K using the silhouette method. Visualize the resulting clustering for in a scatter plot by using different colors for each cluster and also depicting the cluster centroids.

In [ ]:
## TODO Code here....
best_idx = int(np.argmax(scores))
best_k = list(Ks)[best_idx]
best_score = scores[best_idx]
print(f"Optimal K (silhouette): {best_k} (avg silhouette = {best_score:.3f})")


In [ ]:
kmeans_opt = KMeans(n_clusters=best_k, random_state=RANDOM_SEED, n_init="auto")
labels_opt = kmeans_opt.fit_predict(X)
centroids = kmeans_opt.cluster_centers_

plt.figure(figsize=(6, 5))
plt.scatter(X[:, 0], X[:, 1], c=labels_opt, s=50, cmap="tab10")
plt.scatter(centroids[:, 0], centroids[:, 1], s=100, marker="X", c="black")
plt.title(f"KMeans clustering (K={best_k}) with centroids")
plt.xlabel("x1")
plt.ylabel("x2")
plt.show()

## 📢 **HAND-IN** 📢:
1. The plot of the average silhouette scores for different values of K
2. The optimal K you selected based on the plots of the silhouette scores
3. The scatter plot of the clustering results depicting also the centroids for the optimal K

# TASK 2: DBSCAN

First, let's setup the data used in this section. We will redefine the variable `X` containing the data to be clustered.

In [ ]:
from sklearn.datasets import make_moons

In [ ]:
X, _ = make_moons(
    n_samples=200,
    noise=.05,
    random_state=RANDOM_SEED,
)

In [ ]:
plt.scatter(X[:, 0], X[:, 1], s=50)

### Task 2a

* cluster the new data `X` using `KMeans`
* set `n_clusters=2`
* visualize and analyse the resulting clustering
* What do you think of the result?

In [ ]:
kmeans = KMeans(n_clusters=2, random_state=RANDOM_SEED, n_init="auto")
labels = kmeans.fit_predict(X)
centroids = kmeans.cluster_centers_

pad = 0.3
x_min, x_max = X[:,0].min()-pad, X[:,0].max()+pad
y_min, y_max = X[:,1].min()-pad, X[:,1].max()+pad
xx, yy = np.meshgrid(np.linspace(x_min, x_max, 500),
                     np.linspace(y_min, y_max, 500))
grid = np.c_[xx.ravel(), yy.ravel()]

Z = kmeans.predict(grid).reshape(xx.shape)

plt.figure(figsize=(6,5))
plt.contourf(xx, yy, Z, alpha=0.25, cmap="tab10")      # decision regions
plt.contour(xx, yy, Z, levels=[0.5], colors="k", linestyles="--")  # boundary (K=2)

plt.scatter(X[:, 0], X[:, 1], c=labels, s=50, cmap="tab10")
plt.scatter(centroids[:, 0], centroids[:, 1], s=300, marker="X", c="black")
plt.title("KMeans on make_moons (K=2) + centroids")
plt.xlabel("x1"); plt.ylabel("x2")
plt.show()

### Task 2b

Let's use `sklearn.cluster.DBSCAN` instead. Refer to the documentation: <https://scikit-learn.org/stable/modules/generated/sklearn.cluster.DBSCAN.html>

* cluster the data using `DBSCAN`
* try different values for `eps` and `min_samples` until you manage to obtain good clustering of the two half-moons

In [ ]:
from sklearn.cluster import DBSCAN

In [ ]:
eps_list = [0.10, 0.12, 0.15, 0.18, 0.20, 0.22, 0.25, 0.30]
min_samples_list = [3, 5, 8, 10]

for ms in min_samples_list:
    for eps in eps_list:
      db = DBSCAN(eps=eps, min_samples=ms)
      labels = db.fit_predict(X)              # -1 = noise

      n_clusters = len(set(labels)) - (1 if -1 in labels else 0)
      n_noise = np.sum(labels == -1)

      plt.figure(figsize=(5,4))
      plt.scatter(X[:, 0], X[:, 1], c=labels, s=50, cmap="tab10")
      plt.title(f"DBSCAN eps={eps}, min_samples={ms}\nclusters={n_clusters}, noise={n_noise}")
      plt.xlabel("x1")
      plt.ylabel("x2")
      plt.show()

## 📢 **HAND-IN** 📢: No submission needed for this task.

# Task 3 (6 Points): Image Compression - Color Clustering in Images

K-Means can be used for image compression. This task focuses on appling this compression technique to an image.

We provide some useful helper functions below. Read the comments in the code carefully, but do not worry if you don't understand every line.


In [ ]:
from PIL import Image
import requests
from io import BytesIO
import os

In [ ]:
def download_img(url: str) -> Image:
  """
  This function fetches an image from the internet and returns a PIL.Image object
  see: https://pillow.readthedocs.io/en/stable/reference/Image.html

  we tested it mainly on images from wikimedia
  """
  if os.path.exists(url):  # local file
    return Image.open(url).convert("RGB")

  # have to set a fake user-agent so we dont get blocked by wikimedia
  headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.0.0 Safari/537.36'}
  r = requests.get(url, headers=headers)
  if r.status_code != 200:
    # if you hit this exception, consider using another image
    raise Exception(f"download failed:\n{url}")

  return Image.open(BytesIO(r.content)).convert("RGB")

def img2arr(img: Image) -> np.array:
  """
  convert a PIL.Image object to a numpy array
  the resulting array has 3 dimensions [height, width, 3]
  the last dimension contains rgb values

  the rgb values are normalized to be between 0. and 1.
  """
  return np.asarray(img) / 255

def arr2img(arr: np.array) -> Image:
  """
  convert a numpy array back into a PIL.Image object
  we expect the rgb values of the array to be between 0. and 1.
  """
  return Image.fromarray((arr * 255).astype(np.int8), mode="RGB")

In [ ]:
img_url = "https://upload.wikimedia.org/wikipedia/commons/d/d7/RGB_24bits_palette_sample_image.jpg"

img = download_img(img_url)

# you can visualize a PIL.Image object directly in jupyter using `display`
display(img)

img_arr = img2arr(img)

# visualize the np.array version of the same image
plt.imshow(img_arr)

In [ ]:
from typing import Optional

def rg_chromaticity(color_arr: np.array) ->  np.array:
  """
  helper function
  """
  sums = np.sum(color_arr, axis=1, keepdims=True)
  normed = np.divide(color_arr, sums, where=sums > 0.)
  return normed

def rg_chroma_plot(img_arr: np.array, centers: Optional[np.array] = None):
  """
  plot an image in rg-chromaticity space
  this is a 2D representation of 3D rgb data
  refer to wikipedia for details: https://en.wikipedia.org/wiki/Rg_chromaticity

  Note: the resulting plot will not accurately reflect the original euclidean distances

  inputs:
  img_arr: a numpy array with 3 dimensions [height, width, 3] representing an image
  centers: a numpy array with 2 dimensions [n_centers, 3] representing the cluster centers
  """
  colors = np.copy(img_arr).reshape((-1, 3))
  colors = np.unique(colors, axis=0)
  img_rg = rg_chromaticity(colors)
  plt.scatter(img_rg[:, 0], img_rg[:, 1], c=[tuple(colors[i]) for i in range(colors.shape[0])], s=.1)

  if centers is not None:
    crg = rg_chromaticity(centers)
    plt.scatter(crg[:, 0], crg[:, 1], c="black", marker="x", s=25.)

  plt.xlabel("red")
  plt.ylabel("green")
  plt.show()


In [ ]:
rg_chroma_plot(img_arr)

In [ ]:
from sklearn.metrics import pairwise_distances_argmin

def replace_nearest_color(img_arr: np.array, centers: np.array):
  """
  replace each pixel color in `img_arr` by the closest color in `centers`

  input:
  img_arr: a numpy array with 3 dimensions [height, width, 3] representing an image
  centers: a numpy array with 2 dimensions [n_centers, 3] representing the cluster centers
  """
  colors = img_arr.reshape((-1, 3))
  labels = pairwise_distances_argmin(colors, centers)
  compressed = labels.reshape(img_arr.shape[:2])
  replaced = centers[compressed]
  return replaced


In [ ]:
# generate 8 random colors for illustration
random_centers = np.random.default_rng(RANDOM_SEED).random(size=(8, 3))

# plot the random centers on top of the colors of the image
rg_chroma_plot(img_arr, random_centers)

# replace original colors by their nearest neighbors out of the candidate centers
replaced = replace_nearest_color(img_arr, random_centers)

# convert to PIL.Image and visualize
display(arr2img(replaced))

### Task 3 (continued)

* Use an image from [wikimedia](https://commons.wikimedia.org/wiki/Main_Page) to compress. Make sure that it is not too big, e.g. less than 1000px per side

* You can also use the same image we used in the example above

* download the image and convert it to a numpy array using the functions `download_img` and `img2arr` we defined above  

* Cluster the colors in the image using `KMeans`

* Choose the appropriate hyperparameters and verify them (e.g. using `silhouette_score` or the elbow method)

* Visualize your final cluster centers using `rg_chroma_plot` (see example usage above)

* Replace the colors of the original image by their nearest center using `replace_nearest_color` and display the result (see example usage above)

### Tips

* The images have 3 dimensions but the clustering algorithms expect 2. You can use [np.reshape](https://numpy.org/doc/stable/reference/generated/numpy.reshape.html) to obtain the dimension you need.
* To download the images you can right-click and save them to your computer.

In [ ]:
img_url = "/content/Cyclamen_coum_in_december_25-12-2024.__d.j.b_-min_resized.jpg"
img = download_img(img_url)
display(img)

img_arr = img2arr(img)
plt.figure(figsize=(6,4))
plt.imshow(img_arr)
plt.title("Original")
plt.show()

# 2) Reshape image (H,W,3) -> (N,3) for clustering
colors = img_arr.reshape((-1, 3))

In [ ]:
# 3) Sample pixels for model selection (silhouette is expensive on all pixels)
rng = np.random.default_rng(RANDOM_SEED)
sample_size = min(50_000, colors.shape[0])
sample_idx = rng.choice(colors.shape[0], size=sample_size, replace=False)
colors_sample = colors[sample_idx]

In [ ]:
colors_sample = colors

In [ ]:
# 4) Try different K, compute silhouette + inertia (elbow)
Ks = range(2, 33)  # takes a while, so better cut this shorter
sil_scores = []
inertias = []


for k in Ks:
    km = KMeans(n_clusters=k, random_state=RANDOM_SEED)
    labels = km.fit_predict(colors_sample)
    sil_scores.append(silhouette_score(colors_sample, labels))
    inertias.append(km.inertia_)

inertias = np.array(inertias)
delta = inertias[:-1] - inertias[1:]  # improvement per +1 cluster

plt.figure(figsize=(6,4))
plt.plot(list(Ks), inertias, marker="o")
plt.xlabel("K");
plt.ylabel("Inertia");
plt.title("Elbow (fine grid)")
plt.grid(True, alpha=0.3);
plt.show()

plt.figure(figsize=(6,4))
plt.plot(list(Ks)[1:], delta, marker="o")
plt.xlabel("K");
plt.ylabel("Inertia drop from K-1 to K");
plt.title("Marginal gain per extra cluster")
plt.grid(True, alpha=0.3);
plt.show()

# Plot silhouette vs K
plt.figure(figsize=(6,4))
plt.plot(Ks, sil_scores, marker="o")
plt.xticks(Ks)
plt.xlabel("K")
plt.ylabel("Average silhouette score")
plt.title("Silhouette vs K")
plt.grid(True, alpha=0.3)
plt.show()

# Plot elbow (inertia) vs K
plt.figure(figsize=(6,4))
plt.plot(Ks, inertias, marker="o")
plt.xticks(Ks)
plt.xlabel("K")
plt.ylabel("Inertia")
plt.title("Elbow plot (inertia) vs K")
plt.grid(True, alpha=0.3)
plt.show()

In [ ]:
best_k = 2 # considering the elbow method, K = 12 is where the Error is already low, but K isn't unnecessarily large
print("Selected K:", best_k)

kmeans = KMeans(n_clusters=best_k, random_state=RANDOM_SEED)
kmeans.fit(colors_sample)
centers = kmeans.cluster_centers_

# Visualize centers in rg-chromaticity space
rg_chroma_plot(img_arr, centers)

# Compress image: replace each pixel by nearest center
replaced = replace_nearest_color(img_arr, centers)

# Show compressed image
plt.figure(figsize=(6,4))
plt.imshow(replaced)
plt.title(f"Compressed (K={best_k})")
plt.show()

display(arr2img(replaced))

## 📢 **HAND-IN** 📢:

* The original image you used for this task if different than the example image

* The final (compressed) image showing the replaced colors

* Your code for computing the clustering

* Short (2-3 sentences) description on how you verified the clustering and selected the hyperparameters
